# Laboratorium 2

### Przypomnienie implementacji algorytmu Strassena

In [1]:
using LinearAlgebra, DataFrames, BenchmarkTools, GFlops, Statistics

In [2]:
function divide_matrix(M, n)
    """
    :return: Four pieces of Matrix M (divided in half)
    This is an auxiliary function for strassen
    """
    m = div(n, 2)
    A11 = M[1:m, 1:m]
    A12 = M[1:m, m+1:n]
    A21 = M[m+1:n, 1:m]
    A22 = M[m+1:n, m+1:n]

    return A11, A12, A21, A22
end

divide_matrix (generic function with 1 method)

In [3]:
function strassen(A, B)
    """
    :return: C = A * B where A, B, C are matrices
    We can assume that dimensions of A and B are 2 ^ p.
    """
    
    n = size(A, 1)

    if n == 1
        
        if typeof(A) == Vector{Float64} && typeof(B) == Vector{Float64}
            return A .* B
        end
        
        return A * B
    end

    A11, A12, A21, A22 = divide_matrix(A, n)
    B11, B12, B21, B22 = divide_matrix(B, n)

    
    P1 = strassen(A11 + A22, B11 + B22)
    P2 = strassen(A21 + A22, B11)
    P3 = strassen(A11, B12 - B22)
    P4 = strassen(A22, B21 - B11)
    P5 = strassen(A11 + A12, B22)
    P6 = strassen(A21 - A11, B11 + B12)
    P7 = strassen(A12 - A22, B21 + B22)

    C11 = P1 + P4 - P5 + P7
    C12 = P3 + P5
    C21 = P2 + P4
    C22 = P1 - P2 + P3 + P6

    return vcat(hcat(C11, C12), hcat(C21, C22))
end

strassen (generic function with 1 method)

## Rekurencyjne odwracanie macierz


### Opis algorytmu
Algorytm przyjmuję na wejściu macierz. Jego zadaniem jest znaleźć macierz odwrotną do zadanej. Robi to metodą rekurencyjną, zgodnie z wzorami:

#### Macierz A
\begin{bmatrix}
A_{11} & A_{12} \\
A_{21} & A_{22}
\end{bmatrix}


#### Inwersja macierzy A
\begin{bmatrix}
A_{11}^{-1} \left( I + A_{12} S_{22}^{-1} A_{21} A_{11}^{-1} \right) & -A_{11}^{-1} A_{12} S_{22}^{-1} \\
-S_{22}^{-1} A_{21} A_{11}^{-1} & S_{22}^{-1}
\end{bmatrix} 

Gdy macierzy nie da się już podzielić (macierz 1x1) wówczas jej inwersja liczona jest przy pomocy wzoru: $A^{-1} = \frac{1}{A}$

### Implementacja

In [5]:
struct NonSquareMatrixError <: Exception
    message::String
end

function matrix_inverse(A)
    """
    :return matrix B, such that A * B = I
    """
    
    n = size(A, 1)
    m = size(A, 2)
    
    if n != m 
        throw(NonSquareMatrixError("Matrix is not the square"))
    end
    
#     if n == 2
#         m_det = A[1, 1] * A[2, 2] - A[1, 2] * A[2, 1]
#         temp = A[1, 1]
#         A[1, 1] = 1 / m_det * A[2, 2]
#         A[1, 2] = 1 / m_det * (-A[1, 2])
#         A[2, 1] = 1 / m_det * (-A[2, 1])
#         A[2, 2] = 1 / m_det * temp
#         return A
#     end
        
    if n == 1
        return [1 / A[1, 1]]
    end
    
        
    m = div(n, 2)
    A11, A12, A21, A22 = divide_matrix(A, n)
    
    A11_inversed = matrix_inverse(A11)
    S22 = A22 - strassen(strassen(A21, A11_inversed), A12)
    S22_inversed = matrix_inverse(S22)
    
    
    
    B = zeros(n, n)
    eye = Matrix{Float64}(I, div(n, 2) , div(n, 2))
    
    B[1:m, 1:m] = strassen(A11_inversed, eye + strassen(strassen(strassen(A12, S22_inversed), A21), A11_inversed))
    B[1:m, m+1:n] = strassen(strassen(-A11_inversed, A12), S22_inversed)
    B[m+1:n, 1:m] = strassen(strassen(-S22_inversed, A21), A11_inversed)
    B[m+1:n, m+1:n] = S22_inversed
    
    return B
    
end

matrix_inverse (generic function with 1 method)

In [ ]:
# To check corectness
# This implementation may be unstable
eps = 10 ^ -5
min_value = 0.00000001
max_value = 1
for j in 1:3
    for i in 1:8
        A = min_value .+ (max_value - min_value) * rand(2 ^ i, 2 ^ i)
        
        C = matrix_inverse(A)
        
        B = matrix_inverse(C)

        if isapprox(A, B, atol=eps) == false
            println("FALSE!!!", size(A, 1))
            
        end
    end
end

### Pomiary

In [ ]:
min_value = 0.00000001
max_value = 1
inversion_times = DataFrame(size = [], flops= [], time = [])

for i in 1: 10
    for size in 1: 8
        A = min_value .+ (max_value - min_value) * rand(2 ^ size, 2 ^ size)
        m_time = @btime matrix_inverse($A)
        m_flops = @count_ops matrix_inverse($A)
        push!(inversion_times, [size * size, m_flops, m_time])
    end
end     

In [ ]:
inversion_time_group = groupby(inversion_times, [:size])
inversion_time_statistics = combine(inversion_time_group, "time" => mean, "flops" => mean, "time" => std, "flops" => std)

In [ ]:
plot(inversion_time_statistics.size, inversion_time_statistics.time_mean, 
    size = (1000, 2000), year=inversion_time_statistics.time_std,
    yaxis = "TIME[μs]", xaxis = "SIZE")

In [ ]:
plot(inversion_time_statistics.size, inversion_time_statistics.flops_mean, 
    size = (1000, 2000), color="red",year=inversion_time_statistics.flops_std,
    yaxis = "FLOPS", xaxis = "SIZE")